In [1]:

# LIBRARIES GENERAL #
import sys
import os
sys.path.append(os.path.join(os.path.join(os.curdir, '../general_python/')))
sys.path.append(os.path.join(os.path.join(os.curdir, './src/')))

from common.__binary__ import *
from common.__time__ import *
from common.__parsers__ import *
from src.__calculator__ import *

def entropy(  rho,
              TYP = "SCHMIDT"):    
    '''
    Calculate the bipartite entanglement entropy
    '''
    print(TYP)
    entropy = 0
    eV      = None
    if TYP == "SCHMIDT":
        # get schmidt coefficients from singular-value-decomposition
        _, schmidt_coeff, _ = svd(rho)

        # return square
        eV  = np.square(schmidt_coeff)
    else:
        eV  = np.linalg.eigvals(rho)
        
    for i in range(len(eV)):
        entropy += ((-eV[i] * np.log(eV[i])) if (abs(eV[i]) > 0) else 0)
    return entropy

C:\Users\maxgr\anaconda3\Lib\site-packages\tenpy\tools\optimization.py:307: UserWarning: Couldn't load compiled cython code. Code will run a bit slower.
  warnings.warn("Couldn't load compiled cython code. Code will run a bit slower.")


## Spin $1/2$ (all is done in 1D right now)!

### Test the binary implementation of modulo and division operation when one has the spin 1/2 system.

Let us say we have a state 
$$|\underbrace{\sigma _1, ..., \sigma _{m-1}}_{A}, \vert \underbrace{\sigma _m, ..., \sigma _L}_{B} \rangle. $$

The state can be written as an integer. The binary representation of the state is:

$$ s = \sigma _1 * 2^{L-1} + ... + \sigma _L * 2^0. $$

Let us say that the system is divided into $L_A$ and $L_B$ sites. With $|H_A| = 2^{L_A}$ and $|H_B|= 2^{L_B}$: 

a) the L_A subsystem integer is obtainable by dividing $s // |H_A|$. Knowing that the state has specific number of sites, this can be achieved by a bit shift by $L_B$ sites to the right 
$$s >> L_B.$$
b) the $L_B$ subsystem can be obtained by the usage of the mask 

$$m_B = |\underbrace{0, ..., 0}_A, \vert \underbrace{1, ..., 1}_{B} \rangle. = (|H_B| - 1)$$

Of course this can be also done for the second point. 

$$m_A = |\underbrace{1 ..., 1}_A, \vert \underbrace{0, ..., 0}_{B} \rangle. = [(|H_A| - 1) << L_B]$$

In [2]:
La = 12
Lb = 14

dA = 2**La
dB = 2**Lb
rng= np.random.default_rng()

for i in range(1):
    state   = rng.integers(0, dA * dB)

    with Timer() as timer:
        # full state representation
        state_bin = Binary.int2bin(state, La + Lb)
        state_A_bin = state_bin[:La]
        state_B_bin = state_bin[La:]
        state_A_mask = (dA - 1) << Lb
        state_B_mask = dB - 1

        print(f"Full state ({state})=", state_bin)
        print("\tState A =", state_A_bin)
        print("\tState B =", state_B_bin)
        print("\t\tState A mask =", Binary.int2bin(state_A_mask, La + Lb))
        print("\t\tState B mask =", Binary.int2bin(state_B_mask, La + Lb))
        
        # use modulo and division to get the samne
        print("After division and modulo")
        timer.reset()
        print("\tState A =", Binary.int2bin(state // dB, La))
        print("\t\t", "Division took: ", timer.elapsed())
        timer.reset()
        print("\tState B =", Binary.int2bin(state % dB, Lb))
        print("\t\t", "Modulo took: ", timer.elapsed())


        # can one do the same with bit shift, what is faster?
        print("After division and modulo")
        timer.reset()
        print("\tState A =", Binary.int2bin(state >> Lb, La))
        print("\t\t", "Bit shift took: ", timer.elapsed())
        timer.reset()
        print("\tState B =", Binary.int2bin(state & state_B_mask, Lb))
        print("\t\t", "And took: ", timer.elapsed())
        print("--------------------------------------------------------------------------")


Full state (57899784)= 11011100110111101100001000
	State A = 110111001101
	State B = 11101100001000
		State A mask = 11111111111100000000000000
		State B mask = 00000000000011111111111111
After division and modulo
	State A = 110111001101
		 Division took:  0.00014050002209842205
	State B = 11101100001000
		 Modulo took:  0.00014900002861395478
After division and modulo
	State A = 110111001101
		 Bit shift took:  0.0001156999496743083
	State B = 11101100001000
		 And took:  0.00011470000026747584
--------------------------------------------------------------------------


### Extract specific bits from an integer

In [75]:
def extractIntMask(n : int, 
                mask : int):
    result      = 0
    position    = 0
    position_n  = 0
    while mask:
        # Extract the least significant bit of mask
        if mask & 1:
            result      |= ((n >> position) & 1) << position_n
            position_n  += 1
        mask        >>= 1
        position    += 1
    return result

# def extractVecMask(n : int,
#                    mask,
#                    L : int
#                    ):
#     '''
#     Indices have to be sorted from highest to lowest!
#     '''
    
#     # go through the mask and extract the bits
#     result      = 0
    
#     # do first step in moving (remember that mask indicates the positions from left to right,
#     # although the bits are counted from right to left)
    
#     for i, m in enumerate(mask): 
#         # print(f"Moving by {L - mask[position] - 1} bits to the right.")
#         result     |= ((n >> (L - m - 1)) & 1) << i
        
#     return result    
def extractVecMask(n : int,
                   mask,
                   L : int
                   ):
    """
    Indices have to be sorted from highest to lowest!
    """
    # Precompute L - m - 1 values
    shifts = [(L - m - 1) for m in mask]
    
    # Initialize result
    result = 0
    
    # Iterate through mask and extract bits
    for i, m in enumerate(mask):
        # Extract bit from n using bitwise AND
        bit = (n >> shifts[i]) & 1
        # Set bit in result using bitwise OR
        result |= bit << i
    
    return result
def prepareMask(_vec, _size):
    _mask = 0
    for _pos in _vec:
        _mask |= 1 << (_size - 1 - _pos)
    return _mask

# test different mask preparations

bits = 10
n    = np.random.randint(0, 2**bits)
maskL= [1, 2, 5]
maskL.sort(reverse=True)
mask = prepareMask(maskL, bits) 
maskB= Binary.int2bin(mask, bits)

print(f" Mask List of indices = {maskL}\n", 
      f'State Integer = {n}\n',
      f'State string = {Binary.int2bin(n, bits)}\n',
      f"Mask Integer = {mask}\n",
      f"Mask string = {maskB}\n",
      "\tUsing integer mask :", Binary.int2bin(extractIntMask(n, mask), len(maskL)), '\n',
      "\tUsing list mask :", Binary.int2bin(extractVecMask(n, maskL, bits), len(maskL)))



 Mask List of indices = [5, 2, 1]
 State Integer = 878
 State string = 1101101110
 Mask Integer = 400
 Mask string = 0110010000
 	Using integer mask : 100 
 	Using list mask : 100


In [94]:
L  = 12
La = 5
Lb = L-La

dA = 2**La
dB = 2**Lb
d  = (1 << La + Lb)
rng= np.random.default_rng()

state_A_mask    = 0x66 + 1
state_B_mask    = (d - 1) - state_A_mask
print("State A mask =", Binary.int2bin(state_A_mask, La + Lb))
print("State B mask =", Binary.int2bin(state_B_mask, La + Lb))

for i in range(1):
    state   = rng.integers(0, dA * dB)
    with Timer() as timer:
        # full state representation
        state_bin       = Binary.int2bin(state, La + Lb)
        # print states
        elems           = [i for (i, b) in enumerate(Binary.int2bin(state_A_mask, L)) if int(b) == 1]
        elemsR          = [i for (i, b) in enumerate(Binary.int2bin(state_B_mask, L)) if int(b) == 1]
        state_A_bin     = StringParser.ls([state_bin[i] for i in elems], joinElem='', withoutBrcts=True)
        state_B_bin     = StringParser.ls([state_bin[i] for i in range(L) if i not in elems], joinElem='', withoutBrcts=True)
        
        print(f"\tFull state ({state})=", state_bin)
        print("\t\tState A =", state_A_bin)
        print("\t\tState B =", state_B_bin)
        
        timer.reset()
        print("\tState A from my extractor: ", Binary.int2bin(extractIntMask(state, state_A_mask), La))
        print("\t\t", "Bit shift took: ", timer.elapsed())
        timer.reset()
        print("\tState B from my extractor: ", Binary.int2bin(extractIntMask(state, state_B_mask), Lb))
        print("\t\t", "Bit shift took: ", timer.elapsed())
        
        elems.sort(reverse=True) 
        elemsR.sort(reverse=True)        
        timer.reset()
        print("\tState A from my VECTOR extractor: ", Binary.int2bin(extractVecMask(state, elems, L), La))
        print("\t\t", "Bit shift took: ", timer.elapsed())
        timer.reset()
        print("\tState B from my VECTOR extractor: ", Binary.int2bin(extractVecMask(state, elemsR, L), Lb))
        print("\t\t", "Bit shift took: ", timer.elapsed())
        

    print("--------------------------------------------------------------------------")        
        

State A mask = 000001100111
State B mask = 111110011000
	Full state (3336)= 110100001000
		State A = 00000
		State B = 1101001
	State A from my extractor:  00000
		 Bit shift took:  0.00012640003114938736
	State B from my extractor:  1101001
		 Bit shift took:  0.0001962999813258648
	State A from my VECTOR extractor:  00000
		 Bit shift took:  0.000103199970908463
	State B from my VECTOR extractor:  1101001
		 Bit shift took:  8.240004535764456e-05
--------------------------------------------------------------------------


### Schmidt decomposition by bipartite division

$$ |\psi \rangle = \sum _{a, b} \psi _{a,b} |a\rangle |b\rangle \rightarrow |\psi \rangle = \sum _\alpha s _\alpha |\alpha \rangle _A |\alpha \rangle _B $$ 

and $\psi _{a, b}$ is a matrix and indices $a$ go through subsystem $A$ sites and b over subsystem $B$ sites.

In [4]:
L   = 12
La  = 6
Lb  = (L - La)

dA  = int(2**La)
dB  = int(2**Lb)
N   = dA * dB
np.random.seed(0)
psi = genRandomStateCoefficients(dA * dB)
entropy_vonNeuman(psi, L, La, "SCHMIDT"), entropy_vonNeuman(psi, L, La, "EIGENVALUES")

(3.6673804961211403, (3.6673804961211363+1.525548184231068e-17j))

#### Various methods
1. Calculate the reduced density matrix by hand, summing all the contributions with the modulo method
2. Calculate the reduced density matrix by hand, summing all the contributions with the new bitwise method
3. Calculate the reduced density matrix by hand, summing all the contributions with the Schmidt method

In [96]:
with Timer() as timer:
    rho = np.zeros((dA, dA), dtype = complex)
    for n in range(0, N, 1):					
        counter     =   0
        idx         =   n // dB
        s_n_c       =   np.conj(psi[n])
        for m in range(n % dB, N, dB):
            rho[idx, counter]       +=  s_n_c * psi[m]
            counter                 +=  1
    print("Old method took: ", f'{timer.elapsed():.3e}')
    print("\tOld method entropy =", entropy(rho, "OLD"))

#######################################################

state_A_mask = (dA - 1) << Lb
state_B_mask = dB - 1
with Timer() as timer:
    rho = np.zeros((dA, dA), dtype = complex)
    for n in range(0, N, 1):					
        idx         =   n >> Lb
        s_n_c       =   np.conj(psi[n])
        rho[idx]    +=  s_n_c * psi[n & state_B_mask::dB]
    print("Bitwise methodtook: ", f'{timer.elapsed():.3e}')
    print("\tBitwise method entropy =", entropy(rho, "OLD"))

with Timer() as timer:
    rho = psi.reshape(dA, dB)
    print("Schmidt methodtook: ", f'{timer.elapsed():.3e}')
    print("\tSchmidt method entropy =", entropy(rho))


Old method took:  1.812e-01
OLD
	Old method entropy = (3.6673804961211363+1.525548184231068e-17j)
Bitwise methodtook:  1.372e-02
OLD
	Bitwise method entropy = (3.6673804961211407+2.1735336640018013e-17j)
Schmidt methodtook:  8.900e-06
SCHMIDT
	Schmidt method entropy = 3.6673804961211403


#### 

#### Different ordering than half of the system

In [71]:
L   = 16
La  = 3
Lb  = (L - La)

dA  = int(2**La)
dB  = int(2**Lb)
N   = dA * dB
np.random.seed(0)

# string order
state_A_mask    = 7
# left side only (bipartite)
# state_A_mask    = (dA - 1) << Lb
state_B_mask    = N - 1 - state_A_mask
elemsA          = [i for (i, b) in enumerate(Binary.int2bin(state_A_mask, L)) if int(b) == 1]
elemsB          = [i for (i, b) in enumerate(Binary.int2bin(state_B_mask, L)) if int(b) == 1]

print("State A mask =", Binary.int2bin(state_A_mask, La + Lb))
print("State B mask =", Binary.int2bin(state_B_mask, La + Lb))

psi = np.random.normal(size = (dA * dB)) + 1j * np.random.normal(size = dA * dB)
psi = psi / np.linalg.norm(psi)
entropy_vonNeuman(psi, L, La, "SCHMIDT"), entropy_vonNeuman(psi, L, La, "EIGENVALUES")

State A mask = 0000000000000111
State B mask = 1111111111111000


(2.079029578243187, (2.079029578243187-8.652977671715425e-18j))

Method with extracting from integer mask

In [77]:
# try doing Schmidt matrix with a loop
times = []
for i in range(5):
    rho = np.zeros((dA, dB), dtype = complex)
    with Timer() as timer:
        for i, s in enumerate(psi):
            i_A = extractIntMask(i, state_A_mask)
            # print(Binary.int2bin(i, L))
            # print("A=", Binary.int2bin(i_A, La))
            # print("I_A", Binary.int2bin(i_A, La))
            i_B = extractIntMask(i, state_B_mask)
            # print("B=", Binary.int2bin(i_B, Lb))
            # print("-----------------")
            # print("I_B", Binary.int2bin(i_B, Lb))
            # create the Schmidt matrix
            rho[i_A, i_B] += s
        t = timer.elapsed()
        print("The method took: ", f"{t:.3e}", " With S(rho) =", entropy(rho))
        times.append(t)
print(f"{np.mean(times):.2e}, {np.std(times):.2e}")

SCHMIDT
The method took:  3.820e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  3.420e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  3.456e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  3.150e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  2.930e-01  With S(rho) = 2.0788588804033146
3.36e-01, 3.01e-02


Method with extracting from list mask

In [78]:
elemsA.sort(reverse=True)
elemsB.sort(reverse=True)
elemsA, elemsB

([15, 14, 13], [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [80]:
# try doing Schmidt matrix with a loop
times = []
for i in range(5):
    rho = np.zeros((dA, dB), dtype = complex)
    with Timer() as timer:
        for i, s in enumerate(psi):
            i_A = extractVecMask(i, elemsA, L)
            # print("I_A", Binary.int2bin(i_A, La))
            i_B = extractVecMask(i, elemsB, L)
            # print("I_B", Binary.int2bin(i_B, Lb))
            # create the Schmidt matrix
            rho[i_A, i_B] += s
        t = timer.elapsed()
        print("The method took: ", f"{t:.3e}", " With S(rho) =", entropy(rho))
        times.append(t)
print(f"{np.mean(times):.2e}, {np.std(times):.2e}")

SCHMIDT
The method took:  4.353e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  6.438e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  3.470e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  3.739e-01  With S(rho) = 2.0788588804033146
SCHMIDT
The method took:  2.773e-01  With S(rho) = 2.0788588804033146
4.15e-01, 1.25e-01
